In [1]:
### Import Modules
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection

## Import standard installed libraries
#import arcpy
import re
import os
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import datetime as datetime
import numpy as np
import requests
import time
from datetime import timedelta
import json
import sys
import base64

## Set pandas columns to display all
pd.set_option('display.max_columns',None)

In [2]:
gis = GIS("home")

You are logged on as admin_sargeo with an administrator role, proceed with caution.


#### Connect to Layers

In [5]:
### Connect to Team Catalog Layer
catalog_lyr_id = '30958e0139874f87ad26a9ab645216c3' # Team Fact Sheet Layer in Sandbox: https://services.arcgis.com/0ZRg6WRC7mxSLyKX/arcgis/rest/services/survey123_f0c2ab27da8f49e6bf98d28926b416a0/FeatureServer
catalog_lyr_item = gis.content.get(catalog_lyr_id)
catalog_lyr = catalog_lyr_item.layers[0]

cat_name_field = 'task_force_designation'

### Master Layer Connection
master_lyr_id = '97f8571cbc3f4c9e8b92600b253ac544' # Sandbox Master Layer: https://napsg.maps.arcgis.com/home/item.html?id=97f8571cbc3f4c9e8b92600b253ac544
master_lyr_item = gis.content.get(master_lyr_id)
team_name_field = 'team_name'

### Connect to Waypoints
waypoints_lyr = master_lyr_item.layers[0]
### Connect to Worksites
worksites_lyr = master_lyr_item.layers[1]
### Connect to Logistics Points
logpoints_lyr = master_lyr_item.layers[2]
### Connect to Tracklogs
tracklogs_lyr = master_lyr_item.layers[3]

### Structural Hazards Evaluation Form Layer
struct_haz_id = '24d82a5f143747bf88866c0014adf9cc' # Sandbox Structural Hazards layer (Eval Sublayer): https://services.arcgis.com/0ZRg6WRC7mxSLyKX/arcgis/rest/services/Sandbox_v10_Structural_Hazards_Master_-_277b8f_view/FeatureServer/0
struct_haz_item = gis.content.get(struct_haz_id)
struct_haz_lyr = struct_haz_item.layers[0]

#### Functions

In [6]:
def processs_catalog_lst(layer, name_field):

    ### Query in the data from the Catalog layer
    catalog_sdf = layer.query(out_fields=name_field, return_geometry=False).sdf

    ### Sort the Catalog sdf
    names_sorted = catalog_sdf.sort_values(name_field, ascending=False)
 
    ### Deduplicate the Catalog sdf
    deduplicated_df = names_sorted.drop_duplicates(subset=[name_field], keep='first')

    ### Makey list of Team names from deduplicated sdf
    dedupe_lyr_names_list = deduplicated_df[name_field].tolist()

    return dedupe_lyr_names_list

In [20]:
def updateLyrDomain(updatelyr, teamfieldName, catalogNameList):

    lyr_JSON = updatelyr.properties

    # Create a dictionary mapping field names to their indices in the JSON
    field_index_map = {field['name']: index for index, field in enumerate(lyr_JSON['fields'])}

    # Locate the index of the field name in the JSON
    field_name = teamfieldName  # Team Name field

    if field_name in field_index_map:
        field_index = field_index_map[field_name]
        print(f"The index of '{field_name}' is: {field_index}")

        try:
            # Check if the field has a domain and if the domain has coded values
            field_domain = lyr_JSON['fields'][field_index].get('domain')
            if field_domain and 'codedValues' in field_domain:
                field_coded_vals = field_domain['codedValues']
                print("This field has coded domains")

                domains_count = len(catalogNameList)

                if catalogNameList:
                    print(f"{domains_count} domains will be used to update the field: {teamfieldName}...")

                    # Convert the new_list into the desired format for appending to the domains of the update layer
                    formatted_adds_list = [{'name': name, 'code': name} for name in catalogNameList]

                    # Set LastEditDate to ""
                    lyr_JSON["editingInfo"]["lastEditDate"] = ""  # This has to be set to zero/empty string for edits to apply successfully

                    # Sort the coded values alphabetically based on the 'name' key
                    sorted_coded_values = sorted(formatted_adds_list, key=lambda x: x['name'])

                    # Update the JSON with the sorted coded values
                    lyr_JSON['fields'][field_index]['domain']['codedValues'] = sorted_coded_values

                    retry_count = 3  # Number of retries
                    for attempt in range(retry_count):
                        try:
                            # Update the layer's definition with the new coded domains
                            update_result = updatelyr.manager.update_definition(lyr_JSON)
                            print("The layer's definition was updated successfully!")
                            return update_result
                        except Exception as e:
                            print(f"Attempt {attempt + 1} failed: {e}")
                            if attempt < retry_count - 1:
                                print("Retrying...")
                                time.sleep(2)  # Wait before retrying, adjust if needed
                            else:
                                print("All retry attempts failed.")
                                raise e

                else:
                    # Execute this block of code if the list is empty
                    no_domains_message = print(f"There are no new domains from the Team Catalog to add to {updatelyr}.")
                    return no_domains_message

            else:
                print("This field does NOT have a coded domain yet...now what?")
                return None

        except Exception as e:
            print("An error occurred while processing the field's domain.")
            print(e)

    else:
        print(f"Field '{field_name}' not found in the list of fields. Please ensure you have used the correct field name.")


#### Work

In [15]:
cat_names_list = processs_catalog_lst(catalog_lyr, cat_name_field)
print(f'{len(cat_names_list)} Team Names will be used to update the target layers.')

504 Team Names will be used to update the target layers.


In [16]:
### Update Waypoints
waypoints_result_lyr = updateLyrDomain(waypoints_lyr, team_name_field, cat_names_list)

The index of 'team_name' is: 3
This field has coded domains
504 domains will be used to update the field: team_name...
The layer's definition was updated successfully!


In [21]:
### Update Worksites
worksites_result_lyr = updateLyrDomain(worksites_lyr, team_name_field, cat_names_list)

The index of 'team_name' is: 5
This field has coded domains
504 domains will be used to update the field: team_name...
The layer's definition was updated successfully!


In [18]:
### Update Log Points
logpoints_result_lyr = updateLyrDomain(logpoints_lyr, team_name_field, cat_names_list)

The index of 'team_name' is: 3
This field has coded domains
504 domains will be used to update the field: team_name...
The layer's definition was updated successfully!


In [19]:
### Update Tracklogs
tracklogs_result_lyr = updateLyrDomain(tracklogs_lyr, team_name_field, cat_names_list)

The index of 'team_name' is: 3
This field has coded domains
504 domains will be used to update the field: team_name...
The layer's definition was updated successfully!


In [ ]:
### Structure Hazard Layer
#struct_haz_lyr_result = updateLyrDomain(struct_haz_lyr, team_name_field, cat_names_list)